In [1]:
# Based on SF Education's notebook

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats as sps
import scipy.optimize as spop
import matplotlib.pyplot as plt

In [12]:
df = pd.read_csv('../data/BAC_2002-01-01_2022-12-31_1d.csv', index_col=0)
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2001-12-31,19.101077,19.101077,18.822426,18.861378,5397400,0.0,0.0
2002-01-02,18.669628,18.906331,18.609703,18.864384,8669200,0.0,0.0
2002-01-03,18.864382,19.146030,18.804457,18.843409,8237600,0.0,0.0
2002-01-04,18.912315,19.220929,18.879356,19.137033,10598600,0.0,0.0
2002-01-07,18.696586,18.990218,18.696586,18.909319,10852200,0.0,0.0


In [16]:
close = df['Close'].to_frame()
close['rank']=(close['Close'].rank(method='first'))/(len(close))
close = close.reset_index(drop=True)

In [21]:
close.head()

,Close,rank
0,18.861378,0.397882
1,18.864384,0.398071
2,18.843409,0.397315
3,19.137033,0.400530
4,18.909319,0.398449


In [18]:
mean = np.average(close['Close'])
med = np.median(close['Close'])
std = np.std(close['Close'])
var = std**2
skew = sps.skew(close['Close'])
kurt = sps.kurtosis(close['Close'])

In [20]:
print(f'Mean:     {mean}')
print(f'Median:   {med}')
print(f'Std:      {std}')
print(f'Variance: {var}')
print(f'Skew:     {skew}')
print(f'Kurtosis: {kurt}')

Mean:     22.205834364295907
Median:   22.27399253845215
Std:      10.209689196412825
Variance: 104.23775348734875
Skew:     0.1827889739628597
Kurtosis: -0.9625491221002185


In [22]:
# оценим параметры распределения Лапласа методом максимального правдоподобия
loc_laplace = med
scale_laplace = np.average(abs(close['Close']-med))

In [25]:
# оценим параметры распределения Коши методом моментов
loc_cauchy = med
scale_cauchy = (np.percentile(close['Close'], 0.75) - np.percentile(close['Close'], 0.25)) / 2

In [27]:
# оценим параметры распределения Коши методом максимального правдопододия
def cauchy_mle(k):
    return -np.sum(np.log(1/(np.pi*k[1]*(1 + ((close['Close'] - k[0]) / k[1])**2))))

res = spop.minimize(cauchy_mle, [loc_cauchy, scale_cauchy], method='Powell')
loc_cauchy_mle = res.x[0]
scale_cauchy_mle = res.x[1]

f:\python\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [29]:
loc_cauchy_mle, scale_cauchy_mle

(33.843439536037636, -101.41606852206861)